In [40]:
from src.helpers import *
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.utils import to_categorical

from sklearn.metrics import accuracy_score

## read initial data via parser

In [3]:
data_train, data_test = get_parsed_data()
data_train.drop(['date', 'id', 'twitid'], axis=1, inplace = True, errors='ignore')
data_train.fillna(0)

data_train['label'] = data_train.apply(define_label, axis=1)
data_test['label'] = data_test.apply(define_label, axis=1)
data_train['clear_text'] = data_train.text.apply(clear_text)
data_test['clear_text'] = data_test.text.apply(clear_text)

C:\Users\Сергеев\Desktop\DL_task1\src\helpers.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  data_train = pd.concat([df, data_train], ignore_index=True)
C:\Users\Сергеев\Desktop\DL_task1\src\helpers.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  data_test = pd.concat([df, data_test], ignore_index=True)


## feature extraction

In [4]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data_train.text)
X_test = vectorizer.transform(data_test.text)

## sklearn log regression model training and predicting

In [5]:
model = LogisticRegression()
model.fit(X_train, data_train.label)
model.score(X_test, data_test.label)

## vector model training

In [11]:
data = pd.concat([data_train, data_test], ignore_index=True)
split_text = data.clear_text.apply(lambda sent: sent.lower().split())
w2v = Word2Vec(sentences=split_text, sg=1, min_count=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


## transform data to index sequence

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.clear_text)
X_train = tokenizer.texts_to_sequences(data_train.clear_text)
X_test = tokenizer.texts_to_sequences(data_test.clear_text)

In [20]:
X_train = pad_sequences(X_train, maxlen=30)
X_test = pad_sequences(X_test, maxlen=30)

## embeding matrix creation

In [22]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
oov = []
for word, i in tokenizer.word_index.items():
    if word in w2v.wv.vocab:
        embedding_vector = w2v.wv.get_vector(word)
        embedding_matrix[i] = embedding_vector
    else:
        oov.append(word)

In [23]:
NUM = len(list(w2v.wv.vocab.keys()))

## CNN initialization

In [27]:
tweet_input = Input(shape=(30,), dtype='int32')
inp = Embedding(NUM+1, 100, input_length=30,
                weights=[embedding_matrix], trainable=False)(tweet_input)

In [28]:
branches = []

for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
    for i in range(filters_count):
        # Добавляем слой свертки
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='relu')(inp)
        # Добавляем слой субдискретизации
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)
x = concatenate(branches, axis=1) 
drop_1 = Dropout(0.2)(x)
hidden = Dense(30, activation='relu')(drop_1)
drop_3 = Dropout(0.2)(hidden)
out = Dense(3, activation='relu')(drop_3)

In [31]:
model = Model(input=tweet_input, output=out) 

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [32]:
model.fit([X_train], y=to_categorical(data_train.label.as_matrix()+1), verbose=1, epochs=10, batch_size=32)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


Epoch 1/10
18035/18035 [==============================] - 56s 3ms/step - loss: 0.8625 - acc: 0.6487
Epoch 2/10
18035/18035 [==============================] - 50s 3ms/step - loss: 0.7415 - acc: 0.6773
Epoch 3/10
18035/18035 [==============================] - 51s 3ms/step - loss: 0.7239 - acc: 0.6948
Epoch 4/10
18035/18035 [==============================] - 50s 3ms/step - loss: 0.7144 - acc: 0.7047
Epoch 5/10
18035/18035 [==============================] - 52s 3ms/step - loss: 0.6933 - acc: 0.7143
Epoch 6/10
18035/18035 [==============================] - 48s 3ms/step - loss: 0.7034 - acc: 0.7077
Epoch 7/10
18035/18035 [==============================] - 49s 3ms/step - loss: 0.6982 - acc: 0.6983
Epoch 8/10
18035/18035 [==============================] - 49s 3ms/step - loss: 0.6813 - acc: 0.7180
Epoch 9/10
18035/18035 [==============================] - 49s 3ms/step - loss: 0.6643 - acc: 0.7220
Epoch 10/10
18035/18035 [==============================] - 50s 3ms/step - loss: 0.6620 - acc: 0.7254

# Results

In [35]:
model.evaluate(X_test, to_categorical(data_test.label.values+1))

5560/5560 [==============================] - 8s 1ms/step


[0.816078743177781, 0.6366906474820144]

In [44]:
y_ = model.predict(X_test)
y_[-1] = [0,0,1]
to_categorical([np.argmax(x) for x in y_])

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [42]:
from sklearn.metrics import f1_score

In [45]:
f1_score(to_categorical([np.argmax(x) for x in y_]), to_categorical(data_test.label.as_matrix()+1), average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.440960916185547

In [46]:
f1_score(to_categorical([np.argmax(x) for x in y_]), to_categorical(data_test.label.as_matrix()+1), average='micro')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.6368705035971223